In [1]:
import os
import ast
from dotenv import load_dotenv
import arxiv

from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
# hf hub model
hf_hub_llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

hf_chat_model = ChatHuggingFace(llm=hf_hub_llm)

In [ ]:
chat_model = ChatOpenAI( model_kwargs={"stop": ["Observation:"]},)

## Conversation Memories

#### A- ConversationBufferMemory

In [ ]:
prompt = """System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation:
{history}
Human: {input}
AI:"""

history = ''

def talk(question):

    global history
    
    prompt_after_formatting = prompt.format(input=question,
                                            history=history)

    output = chat_model.invoke(prompt_after_formatting)

    history += f"Human: {question}\nAI: {output.content}\n"

    return output.content
    

In [ ]:
talk("How are you today?")
talk("Sorry. What did I ask you?")
talk("Do you know anything about football?")
print(prompt.format(input="current question",history=history))

### B- ConversationSummaryMemory

In [ ]:
prompt = """System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
{summary}
Human: {input}
AI:"""

summary_prompt = """System: Your task is to summarize below conversation with emphasis on key points. If nothing given, return ''.
{history}
Summary:"""

history = ''

def talk(question):

    global history
    
    summary = chat_model.invoke(summary_prompt.format(history=history)).content

    prompt_after_formatting = prompt.format(input=question,
                                            summary=summary)

    output = chat_model.invoke(prompt_after_formatting)

    history += f"Human: {question}\nAI: {output.content}\n"

    return output.content

In [ ]:
talk("How are you today?")

In [ ]:
talk("Sorry. What did I ask you?")

In [ ]:
talk("do you know about football?")

In [ ]:
talk("when did england compete in euros first?")

In [ ]:
talk("what were the other teams in the same competition?")

## Function-Use (Agents and Tools)

In [ ]:
def arxiv_search(args):
    # Construct the default API client.
    client = arxiv.Client()

    # Search for the paper with ID
    search_by_id = arxiv.Search(query=args['query'],
                                id_list=[args['id']])
    # Reuse client to fetch the paper, then print its title.
    first_result = next(client.results(search_by_id))
    
    return first_result.title

def convert_time(args):

    temp = args['time'].split(':')

    return int(temp[0])*3600 + int(temp[1])*60 + int(temp[2])

In [ ]:
system_prompt = """Answer the following questions as best you can. You have access to the following tools:

{tools}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. only one action at a time.
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.

Question: {input}
Thought:{agent_scratchpad}"""

tools = {
    'convert_time': 'A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}',
    'arxiv_search': 'A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}'
}

In [ ]:
history = ''
agent_scratchpad = ''

def ask(question):

    global history, agent_scratchpad

    tools_str = ''

    for func, desc in tools.items():
        tools_str += f"{func} : {desc}\n"
    
    prompt_after_formatting = system_prompt.format(input=question, 
                                                   agent_scratchpad=agent_scratchpad, 
                                                   tools=tools_str, 
                                                   tool_names=list(tools.keys()))
    print(prompt_after_formatting)
    output = chat_model.invoke(prompt_after_formatting)
    print(output.content)
    
    while output.content.find("Final Answer: ")==-1:

        agent_scratchpad += output.content
        #history += f"Human: {question}\nAI: {output.content}\n"

        function_name = output.content.split("Action: ")[1].split('\n')[0]
        function_args = ast.literal_eval(output.content.split("Action Input: ")[1].split('\n')[0])

        result = globals()[function_name](function_args)
        agent_scratchpad += f"Observation: {result}\n"
        
        prompt_after_formatting = system_prompt.format(input=question, 
                                                    agent_scratchpad=agent_scratchpad, 
                                                    tools=tools_str, 
                                                    tool_names=list(tools.keys()))

        output = chat_model.invoke(prompt_after_formatting)
        print(output.content)
    
    return output.content

In [ ]:
print(ask("First answer how many seconds in 4:00:01. Then answer What's the paper 1605.08386 about?"))